## Web Scaping

Pour les sociétés suivantes : Aribus, LVMH et Danone : 

* les ventes au quartier à fin décembre 2018 ;
* le prix de l'action et son % de changement au moment du crawling ;
* le % Shares Owned des investisseurs institutionels ;
* le dividende yield de la company, le secteur et de l'industrie.

In [98]:
import urllib.request as urllib
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [105]:
main_url = "https://www.reuters.com"
search_path = "/finance/stocks/lookup"
financial_path = "/finance/stocks/financial-highlights/"
societes = ['LVMH', 'AirBus', 'Danone']
result = pd.DataFrame(columns=['sales_quater','share_price','share_evol','share_owned','div_company','div_secteur','div_indus'])

In [100]:
def get_soup_from_link(link) :
    page = urllib.urlopen(link)
    soup = BeautifulSoup(page, 'html.parser')
    return soup

In [101]:
def get_symbol(society, link) :
    payload = {'search': society, 'searchType' : 'any'}
    r = requests.get(link, params=payload)
    soup = get_soup_from_link(r.url)
    return soup.find('td', string="Paris Stock Exchange").find_previous_sibling('td').text

In [102]:
def get_info_from_link(link) :
    soup = get_soup_from_link(society_link)
    sales_quater = float(soup.find("td", string='Quarter Ending\xa0Dec-18').parent.find_all("td")[2].text.replace(',',''))
    share_price = float(soup.find("div", class_="sectionQuoteDetail").find_all("span")[1].text.strip())
    share_evol = soup.find("span", class_="valueContentPercent").find("span").text.strip().replace('(','').replace(')','')
    share_owned = soup.find("strong", string="% Shares Owned:").findNext().text
    dividende = soup.find("td", string="Dividend Yield").parent.find_all("td", class_="data")
    div_company = float(dividende[0].text)
    div_secteur = float(dividende[1].text)
    div_indus = float(dividende[2].text)
    return [sales_quater, share_price, share_evol, share_owned, div_company, div_secteur, div_indus]

In [106]:
for societe in societes :
    society_link = main_url + financial_path + get_symbol(societe, main_url + search_path)
    result.loc[societe] = get_info_from_link(society_link)

In [110]:
result

,sales_quater,share_price,share_evol,share_owned,div_company,div_secteur,div_indus
LVMH,13667.7,274.15,-4.04%,20.57%,1.92,1.70,2.60
AirBus,23493.0,102.06,-0.43%,43.53%,1.45,1.34,1.64
Danone,6072.6,65.32,-0.49%,50.60%,2.90,2.78,2.48
